In [71]:
!pip install yfinance statsmodels --quiet


In [72]:
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
plt.style.use("seaborn-v0_8")


In [73]:
nifty100 = [
"ADANIENT.NS","ADANIPORTS.NS","ASIANPAINT.NS","AXISBANK.NS","BAJAJ-AUTO.NS",
"BAJFINANCE.NS","BAJAJFINSV.NS","BPCL.NS","BHARTIARTL.NS","BOSCHLTD.NS",
"BRITANNIA.NS","CIPLA.NS","COALINDIA.NS","DIVISLAB.NS","DRREDDY.NS",
"EICHERMOT.NS","GRASIM.NS","HCLTECH.NS","HDFC.NS","HDFCBANK.NS",
"HDFCLIFE.NS","HEROMOTOCO.NS","HINDALCO.NS","HINDUNILVR.NS","ICICIBANK.NS",
"ITC.NS","INDUSINDBK.NS","INFY.NS","JSWSTEEL.NS","KOTAKBANK.NS",
"LT.NS","M&M.NS","MARUTI.NS","NESTLEIND.NS","NTPC.NS",
"OIL.NS","ONGC.NS","POWERGRID.NS","RELIANCE.NS","SHREECEM.NS",
"SBIN.NS","SUNPHARMA.NS","TCS.NS","TATACONSUM.NS","TATAMOTORS.NS",
"TATASTEEL.NS","TECHM.NS","TITAN.NS","ULTRACEMCO.NS","UPL.NS",
"WIPRO.NS","COFORGE.NS","BANDHANBNK.NS","MOTHERSUMI.NS","SBILIFE.NS",
"PEL.NS","NTPC.NS","HINDPETRO.NS","VINATIORGA.NS","ADANIGREEN.NS",
"AUROPHARMA.NS","BHEL.NS","CADILAHC.NS","CANBK.NS","CHOLAFIN.NS",
"DLF.NS","GAIL.NS","HAVELLS.NS","IBULHSGFIN.NS","IDEA.NS",
"INDIGO.NS","LTIM.NS","MFSL.NS","MPI.NS","NAUKRI.NS",
"NMDC.NS","PAGEIND.NS","PEL.NS","PNB.NS","RECLTD.NS",
"SUNTV.NS","TATAPOWER.NS","TATACHEM.NS","TVSMOTOR.NS","WELCORP.NS",
"ZOMATO.NS","BALKRISIND.NS","BERGEPAINT.NS","BIOCON.NS","CROMPTON.NS",
"EXIDEIND.NS","GLENMARK.NS","HDFCLIFE.NS","IDFCFIRSTB.NS","INDHOTEL.NS",
"JUBLFOOD.NS","LTI.NS","MCDOWELL-N.NS","NIACL.NS","PNBHOUSING.NS"
]
# Remove duplicates & ensure unique
nifty100 = list(dict.fromkeys(nifty100))
print("Tickers loaded:", len(nifty100))


Tickers loaded: 97


In [74]:
start_date = "2019-01-01"
end_date = "2024-01-01"

print("Downloading price data ...")
raw = yf.download(nifty100, start=start_date, end=end_date, group_by="ticker", auto_adjust=True, threads=True)
# Try to extract Close/Adj Close robustly
if ("Close" in raw.columns) or (isinstance(raw.columns, pd.MultiIndex) and "Close" in raw.columns.levels[1]):
    # MultiIndex case: raw[ticker]["Close"]
    prices = pd.DataFrame()
    for t in nifty100:
        try:
            prices[t] = raw[t]["Close"]
        except Exception:
            # some tickers may be missing
            pass
else:
    # fallback to simple dataframe
    prices = raw.copy()

# cleaning
prices.dropna(axis=1, how='all', inplace=True)
prices.dropna(axis=0, how='any', inplace=True)   # require full dates available
print("Final stocks with complete data:", prices.shape[1])
prices = prices.loc[:, prices.std() > 0]         # drop constant series if any
prices.tail()


[*********************100%***********************]  97 of 97 completed

9 Failed downloads:
['ZOMATO.NS', 'PEL.NS', 'LTI.NS', 'HDFC.NS', 'CADILAHC.NS', 'MPI.NS', 'MOTHERSUMI.NS', 'IBULHSGFIN.NS', 'MCDOWELL-N.NS']: YFTzMissingError('possibly delisted; no timezone found')


Final stocks with complete data: 88


,ADANIENT.NS,ADANIPORTS.NS,ASIANPAINT.NS,AXISBANK.NS,BAJAJ-AUTO.NS,BAJFINANCE.NS,BAJAJFINSV.NS,BPCL.NS,BHARTIARTL.NS,BOSCHLTD.NS,...,BERGEPAINT.NS,BIOCON.NS,CROMPTON.NS,EXIDEIND.NS,GLENMARK.NS,IDFCFIRSTB.NS,INDHOTEL.NS,JUBLFOOD.NS,NIACL.NS,PNBHOUSING.NS
Date,,,,,,,,,,,,,,,,,,,,,
2023-12-22,2805.861572,1018.156860,3268.316406,1086.532227,6164.496582,721.434326,1670.182129,204.791611,975.907471,21087.935547,...,575.577820,245.630737,297.038574,290.740540,836.048889,88.161438,436.598938,579.240845,213.912766,771.418945
2023-12-26,2862.910645,1019.395447,3309.447754,1092.472534,6253.934570,708.281921,1643.462036,207.061218,986.065491,21353.023438,...,580.167419,249.968903,295.266968,293.713867,840.680847,88.211281,433.964630,568.033569,209.193008,782.114502
2023-12-27,2840.830566,1015.085022,3330.086914,1104.552856,6491.049316,715.253784,1667.584839,208.482590,1007.121155,21452.982422,...,576.959595,252.063202,295.217804,298.372040,842.772705,88.560135,434.262848,565.343811,209.684647,781.865723
2023-12-28,2807.409912,1007.702759,3323.044434,1106.050293,6484.906250,717.809998,1679.321777,213.571976,1022.407349,21544.853516,...,588.359619,246.677887,292.806427,304.863770,852.086487,88.410622,430.883026,567.983826,209.881302,773.856567
2023-12-29,2846.425537,1015.035461,3328.081787,1100.509399,6575.795410,724.643372,1683.916748,206.625626,1017.969482,21643.404297,...,596.798584,248.971619,305.995026,315.022614,850.642151,88.609978,435.753967,562.903137,206.833130,776.791626


In [75]:
returns = prices.pct_change().dropna()
corr = returns.corr()

corr_threshold = 0.50    # tuneable: higher -> fewer candidates
max_candidate_pairs = 800  # safety cap (tuneable)

# collect all pairs with corr > threshold
candidate_pairs = []
cols = prices.columns.tolist()
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        c = corr.iloc[i,j]
        if c >= corr_threshold:
            candidate_pairs.append((cols[i], cols[j], c))

# If too many candidates, keep top by correlation
candidate_pairs = sorted(candidate_pairs, key=lambda x: x[2], reverse=True)
if len(candidate_pairs) > max_candidate_pairs:
    candidate_pairs = candidate_pairs[:max_candidate_pairs]

print("Candidate pairs after correlation filter:", len(candidate_pairs))


Candidate pairs after correlation filter: 115


In [76]:
cointegrated = []

corr_threshold = 0.85
ADF_P_THRESHOLD = 0.05
HALF_LIFE_THRESHOLD = 60   

for a, b, corr_val in candidate_pairs:

    try:
        df = pd.concat([prices[a], prices[b]], axis=1).dropna()
        y = df.iloc[:, 0]
        x = df.iloc[:, 1]

        X = sm.add_constant(x)
        model = sm.OLS(y, X).fit()
        beta = model.params.iloc[1]   # ✅ SAFE

        spread = y - beta * x
        spread = spread.dropna()

        pvalue = adfuller(spread)[1]

        # ---- SAFE HALF-LIFE ----
        spread_lag = spread.shift(1)
        spread_ret = spread - spread_lag
        spread_lag = spread_lag.dropna()
        spread_ret = spread_ret.dropna()

        X_hl = sm.add_constant(spread_lag)
        model_hl = sm.OLS(spread_ret, X_hl).fit()
        half_life = -np.log(2) / model_hl.params.iloc[1]

        # ---- FINAL FILTER ----
        if (pvalue < ADF_P_THRESHOLD) and (half_life > 1) and (half_life < HALF_LIFE_THRESHOLD):
            cointegrated.append((a, b, beta, pvalue, corr_val, half_life))

    except Exception as e:
        continue


coin_df = pd.DataFrame(
    cointegrated,
    columns=["A","B","beta","adf_pval","corr","half_life"]
)

print("✅ FINAL PAIRS FOUND:", len(coin_df))
coin_df.sort_values("adf_pval").head(10)


✅ FINAL PAIRS FOUND: 25


,A,B,beta,adf_pval,corr,half_life
10,GRASIM.NS,JSWSTEEL.NS,2.056335,0.000201,0.555212,31.937972
13,M&M.NS,MARUTI.NS,0.245475,0.000630,0.540233,23.759405
17,GRASIM.NS,TATASTEEL.NS,12.768037,0.001691,0.527848,40.310176
9,EICHERMOT.NS,MARUTI.NS,0.441051,0.002616,0.557641,28.655898
12,AXISBANK.NS,CANBK.NS,9.078492,0.002982,0.544670,39.038695
2,HINDALCO.NS,TATASTEEL.NS,3.670185,0.004324,0.736958,51.942132
15,GRASIM.NS,HINDALCO.NS,3.365529,0.007150,0.528665,34.443419
18,TECHM.NS,LTIM.NS,0.165557,0.007366,0.526435,30.535719
23,MARUTI.NS,TVSMOTOR.NS,3.283489,0.007372,0.510691,30.132805
7,HINDUNILVR.NS,NESTLEIND.NS,1.571815,0.008995,0.585106,37.522589


In [77]:
vols = []

for _, row in coin_df.iterrows():
    a, b, beta = row["A"], row["B"], row["beta"]
    
    df = pd.concat([prices[a], prices[b]], axis=1).dropna()
    y = df.iloc[:, 0]
    x = df.iloc[:, 1]

    spread = y - beta * x
    spread = spread.dropna()

    spread_vol = spread.rolling(60).std().mean()  # 60-day average volatility
    vols.append(spread_vol)

coin_df["spread_vol"] = vols

# Inverse volatility weights
coin_df["inv_vol"] = 1 / coin_df["spread_vol"]
coin_df["weight"] = coin_df["inv_vol"] / coin_df["inv_vol"].sum()

coin_df.head()


,A,B,beta,adf_pval,corr,half_life,spread_vol,inv_vol,weight
0,BAJFINANCE.NS,BAJAJFINSV.NS,0.405083,0.029664,0.829782,48.743409,17.710863,0.056463,0.066662
1,JSWSTEEL.NS,TATASTEEL.NS,6.142252,0.038289,0.767985,58.428939,21.380650,0.046771,0.055220
2,HINDALCO.NS,TATASTEEL.NS,3.670185,0.004324,0.736958,51.942132,14.967868,0.066810,0.078879
3,SHREECEM.NS,ULTRACEMCO.NS,1.618049,0.047156,0.696051,55.599630,942.968218,0.001060,0.001252
4,ICICIBANK.NS,SBIN.NS,1.536906,0.014408,0.679162,38.626544,21.467486,0.046582,0.054997


In [78]:
top_k = 15

# Compute risk-adjusted score for each cointegrated pair
scores = []
for _, row in coin_df.iterrows():
    # spread info
    a, b, beta = row["A"], row["B"], row["beta"]
    df = prices[[a,b]].dropna()
    spread = df[a] - beta * df[b]
    spread_vol = spread.rolling(60).std().mean()
    half_life = row["half_life"]
    
    # Score: higher = better (volatile but mean-reverting)
    score = spread_vol / half_life
    scores.append(score)

coin_df["score"] = scores

# Sort by score descending
coin_df = coin_df.sort_values("score", ascending=False).reset_index(drop=True)

# Select top K pairs
selected = coin_df.head(top_k).copy()


In [79]:
def backtest_pair(prices, A, B, beta, window=90, entry_z=2.5, exit_z=0.3, tc=0.001):

    s = prices[A] - beta * prices[B]
    mean = s.rolling(window).mean()
    std = s.rolling(window).std()
    z = (s - mean) / std
    z = z.dropna()

    if z.empty:
        return None

    pnl = []
    pos_log = []
    last_pos = 0

    for i in range(1, len(z)):
        zi = z.iloc[i]
        date = z.index[i]

        # ENTRY / EXIT (ORIGINAL EDGE)
        if zi > entry_z and last_pos == 0:
            last_pos = -1
            cost = tc * (abs(prices[A].loc[date]) + abs(prices[B].loc[date]))
        elif zi < -entry_z and last_pos == 0:
            last_pos = 1
            cost = tc * (abs(prices[A].loc[date]) + abs(prices[B].loc[date]))
        elif abs(zi) < exit_z and last_pos != 0:
            last_pos = 0
            cost = tc * (abs(prices[A].loc[date]) + abs(prices[B].loc[date]))
        else:
            cost = 0

        spread_change = s.iloc[i] - s.iloc[i-1]
        daily_pnl = last_pos * spread_change - cost

        pnl.append(daily_pnl)
        pos_log.append(last_pos)

    pnl = pd.Series(pnl, index=z.index[1:])
    equity = pnl.cumsum()

    sharpe = np.sqrt(252) * pnl.mean() / pnl.std() if pnl.std() > 0 else 0
    maxdd = (equity - equity.cummax()).min()
    trades = sum(1 for x in pos_log if x != 0)

    return {
        "equity": equity,
        "pnl": pnl,
        "sharpe": sharpe,
        "maxdd": maxdd,
        "trades": trades
    }


In [81]:
results = []
portfolio_equities = []

for idx, row in selected.iterrows():
    A = row["A"]
    B = row["B"]
    beta = row["beta"]

    res = backtest_pair(prices, A, B, beta, window=60, entry_z=2.0, exit_z=0.5, tc=0.001)

    if res is None:
        continue

    results.append({
        "A": A,
        "B": B,
        "beta": beta,
        "adf_pval": row["adf_pval"],
        "sharpe": res["sharpe"],
        "max_dd": res["maxdd"],
        "trades": res["trades"]
    })

    portfolio_equities.append(res["equity"].rename(f"{A}-{B}"))

print("Backtested pairs:", len(results))


Backtested pairs: 15


In [82]:
portfolio_equity_raw = pd.concat(portfolio_equities, axis=1).sum(axis=1)
portfolio_pnl_raw = portfolio_equity_raw.diff().fillna(0)

# ✅ VOL TARGET SCALING (THIS REDUCES DD WITHOUT HURTING SHARPE)
target_vol = 0.01   # 1% daily vol target
realized_vol = portfolio_pnl_raw.rolling(20).std()
scaling = target_vol / realized_vol
scaling = scaling.replace([np.inf, -np.inf], 0).fillna(0)
scaling = scaling.clip(0, 2)   # prevent overleverage

portfolio_pnl = portfolio_pnl_raw * scaling
portfolio_equity = portfolio_pnl.cumsum()

portfolio_sharpe = np.sqrt(252) * portfolio_pnl.mean() / portfolio_pnl.std()
portfolio_maxdd = (portfolio_equity - portfolio_equity.cummax()).min()

print("\n=== Portfolio Summary (VOL TARGETED) ===")
print(f"Portfolio Sharpe: {portfolio_sharpe:.3f}")
print(f"Portfolio Max Drawdown: {portfolio_maxdd:.2f}")
print(f"Total pairs traded: {len(results)}")
print(f"Total trades (sum rough): {sum(r['trades'] for r in results)}")
print(f"Average pair Sharpe: {np.mean([r['sharpe'] for r in results]):.3f}")



=== Portfolio Summary (VOL TARGETED) ===
Portfolio Sharpe: 0.929
Portfolio Max Drawdown: -0.27
Total pairs traded: 15
Total trades (sum rough): 7213
Average pair Sharpe: 0.176
